In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

## Model

In [54]:
usecols = ['display_id', 'ad_id', 'clicksPerShows', 'dist', 'dist_cats', 'dist_ents', 'clicked']
features = ['clicksPerShows', 'dist2']
#features = ['p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
features_asis = []
intercept = True
class_weight = None

In [3]:
def calc_features(data):
    data['dist2'] = np.square(data['dist'])
    data.drop('dist', axis=1, inplace=True)
    data['dist_ents2'] = np.square(data['dist_ents'])
    data['dist_ents3'] = data['dist_ents2']*data['dist_ents']
    data.drop('dist_ents', axis=1, inplace=True)
    return data

In [4]:
filename_train = "../generated/final/events_clicks_CTR_addocs_dist_time_train.csv"
pd.read_csv(filename_train, nrows=10).head()

,display_id,document_id,timestamp,traffic_source,platform,day,hour,geo,ad_id,clicked,adsPerDisplay,clicksPerShows,addoc_id,dist,dist_cats,dist_ents,doc_time,addoc_time
0,8,1330329,638,1.0,2.0,1,7,2765,95724,0,4,0.034785,1136820,0.537786,1.254512,1.075178,-3.860280e+10,-8.661600e+09
1,8,1330329,638,1.0,2.0,1,7,2765,175694,0,4,0.202327,1392479,0.430210,0.098995,1.073816,-3.860280e+10,-3.045600e+09
2,8,1330329,638,1.0,2.0,1,7,2765,280430,1,4,0.256853,1601512,0.245120,1.304837,NaN,-3.860280e+10,NaN
3,8,1330329,638,1.0,2.0,1,7,2765,329774,0,4,0.052426,1652347,0.259652,1.192209,0.702294,-3.860280e+10,-6.264000e+08
4,2657,1271490,185255,1.0,2.0,1,7,2765,70081,0,4,0.150145,933594,0.124739,0.409647,NaN,-5.839200e+09,-3.242160e+10


In [5]:
pd.read_csv(filename_train, nrows=10, usecols=usecols).head()

,display_id,ad_id,clicked,clicksPerShows,dist,dist_cats,dist_ents
0,8,95724,0,0.034785,0.537786,1.254512,1.075178
1,8,175694,0,0.202327,0.430210,0.098995,1.073816
2,8,280430,1,0.256853,0.245120,1.304837,NaN
3,8,329774,0,0.052426,0.259652,1.192209,0.702294
4,2657,70081,0,0.150145,0.124739,0.409647,NaN


In [6]:
def join(series):
    #return ' '.join( str(list(series)).strip('[]').split(', ') )
    return ' '.join( map(str, series) )

In [7]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

In [8]:
def calc_min_max(data):
    d={}
    for feature in features:
        if not(feature in features_asis):
            d[feature+'_min'] = data.groupby('display_id')[feature].min()
            d[feature+'_max'] = data.groupby('display_id')[feature].max()
    
    dataM = pd.DataFrame(d).reset_index()
    #dataM.drop(features, axis=1 ,inplace=True)
    return dataM

In [9]:
def fill_na(data, na_list):
    d = {}
    for feature in na_list:
        d[feature] = data[feature].mean()
    return data.fillna(d)

In [10]:
def calc_l(dataM):
    for feature in features:
        if not(feature in features_asis):
            dataM[feature+'_l'] = dataM[feature + '_max'] - dataM[feature + '_min'] 
            dataM.loc[dataM[feature+'_l'] == 0, feature+'_l'] = 1
            dataM.drop([feature+'_max'], axis=1, inplace=True)
    return dataM    

In [11]:
def data_norm(data):
    for feature in features:
        if not(feature in features_asis):
            data[feature] = ( data[feature] - data[feature+'_min'] ) / data[feature+'_l']
            data.drop([feature+'_min', feature+'_l'], axis=1, inplace=True) 
    return data

## Preparation for Validation

#### Step 0

In [8]:
train_nrows, test_nrows =  None, None # 650000, 210000 #

In [9]:
train_index = pd.read_csv("../generated/final/train_index.csv", nrows=train_nrows)['index'].ravel()
test_index = pd.read_csv("../generated/final/test_index.csv", nrows=test_nrows)['index'].ravel()

In [10]:
nrows = max(train_index[-1], test_index[-1])+1
nrows

87141731

In [11]:
train = pd.read_csv(filename_train, usecols = usecols, nrows=nrows)
train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
clicksPerShows    87141731
dist              77941525
dist_cats         83237252
dist_ents         42904726
dtype: int64

In [12]:
train = fill_na(train, ['dist', 'dist_cats', 'dist_ents'])
train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
clicksPerShows    87141731
dist              87141731
dist_cats         87141731
dist_ents         87141731
dtype: int64

In [13]:
train.to_csv("../generated/solutions/10_step0.csv", index=False)

In [14]:
del train

#### Step 1

In [17]:
output_file = "../generated/solutions/10_step1.csv"

In [18]:
f = open(output_file, 'w')
first = True
for train in pd.read_csv("../generated/solutions/10_step0.csv", chunksize=10**7):
    train = calc_features(train)
    
    if first: 
        first = False 
        train.to_csv(f, header=True, index=False)
        f.close()
        f = open(output_file, 'a')
    else:
        train.to_csv(f, header=False, index=False)
        
f.close()
print 'Done'

Done


#### Step 2

In [19]:
train = pd.read_csv("../generated/solutions/10_step1.csv")
train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
clicksPerShows    87141731
dist_cats         87141731
dist2             87141731
dist_ents2        87141731
dist_ents3        87141731
dtype: int64

In [20]:
train.ix[test_index].to_csv("../generated/solutions/10_step2_test.csv", index=False)
train.ix[train_index].to_csv("../generated/solutions/10_step2_train.csv", index=False)
del train

#### Step 3

In [12]:
train = pd.read_csv("../generated/solutions/10_step2_train.csv")

In [13]:
trainM = calc_min_max(train)
#trainM.count()

In [14]:
trainM = calc_l(trainM)
#trainM.count()

In [16]:
output_file = "../generated/solutions/10_step3_train.csv"
f = open(output_file, 'w')
first = True
for train in pd.read_csv("../generated/solutions/10_step2_train.csv", chunksize=10**7):
    train = train.merge(trainM, on='display_id')
    train = data_norm(train)
    
    if first:
        first=False
        train.to_csv(f, header=True, index=False)
        f.close()
        f = open(output_file, 'a')
    else:
        train.to_csv(f, header=False, index=False)

f.close()

In [17]:
del trainM
del train

In [18]:
test = pd.read_csv("../generated/solutions/10_step2_test.csv")

In [19]:
testM = calc_min_max(test)
testM = calc_l(testM)

In [20]:
test = test.merge(testM, on='display_id')
del testM

In [21]:
test = data_norm(test)
#test.count()

In [22]:
test.to_csv("../generated/solutions/10_step3_test.csv", index=False)

#### Step 4

In [23]:
train = pd.read_csv("../generated/solutions/10_step3_train.csv")
test = pd.read_csv("../generated/solutions/10_step3_test.csv")

In [24]:
scaler = preprocessing.StandardScaler()
excpt = [] 
for f in features:
    if not f in excpt:
        print f
        scaler.fit(train[[f]])
        train[f] = scaler.transform(train[[f]]) 
        test[f] = scaler.transform(test[[f]])

clicksPerShows
dist2
dist_cats
dist_ents2
dist_ents3


In [26]:
train.to_csv("../generated/solutions/10_step4_train.csv", index=False)
test.to_csv("../generated/solutions/10_step4_test.csv", index=False)

In [25]:
train[features].head()

,clicksPerShows,dist2,dist_cats,dist_ents2,dist_ents3
0,-1.123144,1.624163,0.882414,2.054183,2.119184
1,0.852239,0.464969,-1.305112,2.042145,2.104633
2,1.495124,-0.926478,0.977686,2.051900,2.116423
3,-0.915151,-0.844819,0.764467,-0.672020,-0.646231
4,-0.050373,-0.926478,-1.305112,-0.672020,-0.646231


## Validation

In [55]:
clf = linear_model.SGDClassifier(random_state=0, loss='log', fit_intercept=intercept, n_iter=5, class_weight=class_weight)
clf.fit(train[features], train['clicked'].ravel())

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

In [56]:
clf.classes_

array([0, 1], dtype=int64)

In [57]:
clicks_test_sample = test[['display_id', 'ad_id', 'clicked']]
clicks_test_sample.count()

display_id    21786490
ad_id         21786490
clicked       21786490
dtype: int64

In [58]:
validation = clicks_test_sample[clicks_test_sample['clicked']==1].astype({'ad_id':str}).sort_values('display_id')
validation.count()

display_id    4220025
ad_id         4220025
clicked       4220025
dtype: int64

In [59]:
test[features].head()

,clicksPerShows,dist2
0,1.495124,-0.926478
1,-1.123144,1.022875
2,-1.005127,0.241676
3,0.731841,-0.374877
4,-0.956287,1.624163


In [60]:
test['res'] = pd.Series(clf.predict_proba(test[features])[:,1])
test.head()

,display_id,ad_id,clicked,clicksPerShows,dist_cats,dist2,dist_ents2,dist_ents3,res
0,19972,120162,1,1.495124,-1.305112,-0.926478,-0.67202,-0.646231,0.421557
1,19972,122279,0,-1.123144,-1.305112,1.022875,-0.67202,-0.646231,0.062410
2,19972,168342,0,-1.005127,-1.305112,0.241676,-0.67202,-0.646231,0.068310
3,19972,176585,0,0.731841,-1.305112,-0.374877,-0.67202,-0.646231,0.266134
4,19972,250080,0,-0.956287,-1.305112,1.624163,-0.67202,-0.646231,0.072742


In [61]:
clf.coef_, clf.intercept_[0]

(array([[ 0.92609711,  0.01619549]]), -1.6860087543971871)

In [62]:
prediction = test.sort_values(by=['display_id', 'res'], ascending=[True, False])
prediction.count()

display_id        21786490
ad_id             21786490
clicked           21786490
clicksPerShows    21786490
dist_cats         21786490
dist2             21786490
dist_ents2        21786490
dist_ents3        21786490
res               21786490
dtype: int64

In [63]:
prediction=pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).astype({'ad_id':str}).reset_index()
prediction.count()

display_id    4220025
ad_id         4220025
dtype: int64

In [64]:
prediction.head()

,display_id,ad_id
0,1,279295 144739 42337 139684 296965 156824
1,7,300808 215967 105766
2,9,151028 19959 140940 104208
3,14,98270 143467 224171 288396
4,26,152193 285992


In [65]:
validation.head()

,display_id,ad_id,clicked
18142714,1,144739,1
14412118,7,105766,1
11270842,9,140940,1
377269,14,224171,1
18233315,26,152193,1


In [66]:
err = model_evaluation(validation, prediction)  

In [67]:
print 'clicksPerShows :', err  #0.662869594202      0.589962163742     

clicksPerShows : 0.662851376673


In [48]:
#['clicksPerShows']                             0.662869594202       0.588499968978 
#['clicksPerShows', 'dist']                     0.66282443126        0.588638094838     -> 0.662839173432 (niter=20)
                                                                    #0.588785527156
# ['clicksPerShows', 'dist', 'dist_cats']       0.663048972212       0.589767768832    
# ['clicksPerShows', 'dist_cats']                                    0.589561035359
# ['clicksPerShows', 'dist2', 'dist_cats']                           0.589826269431
# [1, 'clicksPerShows', 'dist2']                0.662851376673  
# [..., 'dist2', 'dist_cats']                   0.663003642753       0.589922876476 
# [..., 'dist2', 'dist_cats', 'dist_ents']                           0.589944691616 
# [..., 'dist2', 'dist_cats', 'dist_ents2']                          0.589952567082 
# [..., 'dist_cats', 'dist_ents', 'dist_ents2']                      0.589962163742 
# [..., 'dist_cats', 'dist_ents2', 'dist_ents3'] 0.662808371039      0.590029410411 <<<<<<<<<<
    
                                      #         0.6628185045   +platform
                                     #          0.662806944913 +intercept
#['clicksPerShows', 'dist', 'adsPerDisplay']                         0.588542728654                                     
#['clicksPerShows', 'p1', 'p2', 'p3', 'dist']                        0.588165686987
#['clicksPerShows', 'dist', 't1', 't2', 't3']                        0.588181057654
#['clicksPerShows', 'p1', 'p2', 'p3']                                0.588499968978
#['clicksPerShows', 't1', 't2', 't3']                                0.588499968978 
#['clicksPerShows', 'adsPerDisplay']                                 0.588499968978


In [49]:
print err

0.590029410411
